In [1]:
import pyforest

In [2]:
df = pd.read_csv('data.csv')
df.dropna(subset=['Body'], inplace = True)
df.head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3988 entries, 0 to 4008
Data columns (total 4 columns):
URLs        3988 non-null object
Headline    3988 non-null object
Body        3988 non-null object
Label       3988 non-null int64
dtypes: int64(1), object(3)
memory usage: 155.8+ KB


In [4]:
duplicates = df[df.URLs.duplicated(keep = False)].sort_values(by='URLs')
duplicates.reset_index(inplace = True)
duplicates.drop(['index'], axis = 1, inplace= True)
duplicates.head(10)

,URLs,Headline,Body,Label
0,http://beforeitsnews.com/entertainment/2017/09...,10 Shocking Facts About Porn You Probably Didn...,"Vietnam Is in Great Danger, You Must Publish a...",0
1,http://beforeitsnews.com/entertainment/2017/09...,10 Shocking Facts About Porn You Probably Didn...,Shooter's Hotel Room: Bad News After What Was ...,0
2,http://beforeitsnews.com/entertainment/2017/09...,10 Shocking Facts About Porn You Probably Didn...,An Embattled Pharmaceutical Company That Sells...,0
3,http://beforeitsnews.com/entertainment/2017/09...,10 Shocking Facts About Porn You Probably Didn...,Pink Floyd comfortably numb! Not for long but ...,0
4,http://beforeitsnews.com/entertainment/2017/10...,10 Awful Moments Caught on Video!,10 Awful Moments Caught on Video!\n% of reader...,0
5,http://beforeitsnews.com/entertainment/2017/10...,10 Awful Moments Caught on Video!,Red Flag Warning: These California Wildfires A...,0
6,http://beforeitsnews.com/entertainment/2017/10...,LA Releases First Single SHOWTIME,A Potato Battery Can Light up a Room for Over ...,0
7,http://beforeitsnews.com/entertainment/2017/10...,LA Releases First Single SHOWTIME,"Vietnam Is in Great Danger, You Must Publish a...",0
8,http://beforeitsnews.com/entertainment/2017/10...,LA Releases First Single SHOWTIME,An Embattled Pharmaceutical Company That Sells...,0
9,http://beforeitsnews.com/scandals/2017/09/fema...,FEMA Is a Front for a Depop Program? - FEMA Sh...,Pink Floyd comfortably numb! Not for long but ...,0


In [5]:
df.drop_duplicates(subset = 'Body', keep = False, inplace = True)

In [6]:
for i in range(len(duplicates.index)-1):
    if duplicates.URLs[i] == duplicates.URLs[i + 1] and duplicates.Body[i][:30] == duplicates.Body[i+1][:30]:
        duplicates.drop(i, axis = 0, inplace = True)

In [7]:
duplicates.head()

,URLs,Headline,Body,Label
0,http://beforeitsnews.com/entertainment/2017/09...,10 Shocking Facts About Porn You Probably Didn...,"Vietnam Is in Great Danger, You Must Publish a...",0
1,http://beforeitsnews.com/entertainment/2017/09...,10 Shocking Facts About Porn You Probably Didn...,Shooter's Hotel Room: Bad News After What Was ...,0
2,http://beforeitsnews.com/entertainment/2017/09...,10 Shocking Facts About Porn You Probably Didn...,An Embattled Pharmaceutical Company That Sells...,0
3,http://beforeitsnews.com/entertainment/2017/09...,10 Shocking Facts About Porn You Probably Didn...,Pink Floyd comfortably numb! Not for long but ...,0
4,http://beforeitsnews.com/entertainment/2017/10...,10 Awful Moments Caught on Video!,10 Awful Moments Caught on Video!\n% of reader...,0


In [8]:
len(duplicates)

783

In [9]:
clean_df = pd.concat([df, duplicates])
clean_df.reset_index(inplace = True)
clean_df.drop(['index'], axis = 1, inplace = True)
clean_df.head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,https://www.nytimes.com/2017/10/10/us/politics...,Kansas Tried a Tax Plan Similar to Trump’s. It...,"In 2012, Kansas lawmakers, led by Gov. Sam Bro...",1


In [10]:
nlp = spacy.load('en_core_web_lg')

In [25]:
clean_df['token_body'] = [ nlp(clean_df['Body'][i]) for i in range(0, len(clean_df))]

In [26]:
clean_df.head()

,URLs,Headline,Body,Label,token_body
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1,"(Image, copyright, Getty, Images, \n, On, Sund..."
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1,"(LONDON, (, Reuters, ), -, “, Last, Flag, Flyi..."
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1,"(The, feud, broke, into, public, view, last, w..."
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1,"(MEXICO, CITY, (, Reuters, ), -, Egypt, ’s, Ch..."
4,https://www.nytimes.com/2017/10/10/us/politics...,Kansas Tried a Tax Plan Similar to Trump’s. It...,"In 2012, Kansas lawmakers, led by Gov. Sam Bro...",1,"(In, 2012, ,, Kansas, lawmakers, ,, led, by, G..."


In [27]:
clean_df['token_headline'] = [ nlp(clean_df['Headline'][i]) for i in range(0, len(clean_df))]

In [28]:
clean_df.head()

,URLs,Headline,Body,Label,token_body,token_headline
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1,"(Image, copyright, Getty, Images, \n, On, Sund...","(Four, ways, Bob, Corker, skewered, Donald, Tr..."
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1,"(LONDON, (, Reuters, ), -, “, Last, Flag, Flyi...","(Linklater, 's, war, veteran, comedy, speaks, ..."
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1,"(The, feud, broke, into, public, view, last, w...","(Trump, ’s, Fight, With, Corker, Jeopardizes, ..."
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1,"(MEXICO, CITY, (, Reuters, ), -, Egypt, ’s, Ch...","(Egypt, 's, Cheiron, wins, tie, -, up, with, P..."
4,https://www.nytimes.com/2017/10/10/us/politics...,Kansas Tried a Tax Plan Similar to Trump’s. It...,"In 2012, Kansas lawmakers, led by Gov. Sam Bro...",1,"(In, 2012, ,, Kansas, lawmakers, ,, led, by, G...","(Kansas, Tried, a, Tax, Plan, Similar, to, Tru..."
